In [ ]:
pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 3.4 MB/s eta 0:00:00


In [ ]:
from pandas.api.types import CategoricalDtype

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
import xlsxwriter
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from openpyxl import load_workbook

## Data Transformation

In [ ]:
link = "https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx"
df1 = pd.read_excel(link, sheet_name = "Year 2009-2010")
df1.set_axis(['Invoice', 'Stock Code', 'Description', 'Quantity', 'Invoice Date', 'Price', 'Customer ID', 'Country'], axis='columns', inplace=True)

<ipython-input-5-8808ed51a9bc>:3: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df1.set_axis(['Invoice', 'Stock Code', 'Description', 'Quantity', 'Invoice Date', 'Price', 'Customer ID', 'Country'], axis='columns', inplace=True)


In [ ]:
link = "https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx"
df2 = pd.read_excel(link, sheet_name = "Year 2010-2011")
df2.set_axis(['Invoice', 'Stock Code', 'Description', 'Quantity', 'Invoice Date', 'Price', 'Customer ID', 'Country'], axis='columns', inplace=True)

<ipython-input-6-f8b456e121d2>:3: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df2.set_axis(['Invoice', 'Stock Code', 'Description', 'Quantity', 'Invoice Date', 'Price', 'Customer ID', 'Country'], axis='columns', inplace=True)


In [ ]:
# Melihat jumlah nilai kosong pada setiap dataframe
print("Data tahun 2009-2010")
print(df1.isnull().sum())
print("\n")
print("Data tahun 2010-2011")
print(df2.isnull().sum())

Data tahun 2009-2010
Invoice              0
Stock Code           0
Description       2928
Quantity             0
Invoice Date         0
Price                0
Customer ID     107927
Country              0
dtype: int64


Data tahun 2010-2011
Invoice              0
Stock Code           0
Description       1454
Quantity             0
Invoice Date         0
Price                0
Customer ID     135080
Country              0
dtype: int64


Pada data tahun 2009-2010, terdapat 2.928 *missing values* pada variabel "Description", dan 107.927 *missing values* pada variabel "Customer ID". Sedangkan di data tahun 2010-2011, terdapat 1.454 *missing values* pada variabel "Description", dan 135.080 *missing values* pada variabel "Customer ID".

In [ ]:
def dropNull(df):
  df["Description"] = df["Description"].astype('string')
  df["Customer ID"] = df["Customer ID"].astype('string')

  df["Description"].fillna("empty", inplace = True)
  df["Customer ID"].fillna("empty", inplace = True)

  df["DESC"] = list(map(lambda x: x.islower(), df['Description']))
  df["CUST"] = list(map(lambda x: x.islower(), df['Customer ID']))

  df_filtered1 = df[df['DESC'] == True].index
  df.drop(df_filtered1, inplace = True)
  df.drop('DESC', inplace = True, axis = 1)

  df_filtered2 = df[df['CUST'] == True].index
  df.drop(df_filtered2, inplace = True)
  df.drop('CUST', inplace = True, axis = 1)

In [ ]:
# Menghilangkan missing value dari dataframe
dropNull(df1)
dropNull(df2)

In [ ]:
# Hasil dari pembersihan data
print("Data tahun 2009-2010")
print(df1.isnull().sum())
print("\n")
print("Data tahun 2010-2011")
print(df2.isnull().sum())

Data tahun 2009-2010
Invoice         0
Stock Code      0
Description     0
Quantity        0
Invoice Date    0
Price           0
Customer ID     0
Country         0
dtype: int64


Data tahun 2010-2011
Invoice         0
Stock Code      0
Description     0
Quantity        0
Invoice Date    0
Price           0
Customer ID     0
Country         0
dtype: int64


Dapat dilihat bahwa fungsi yang dibuat sudah berhasil menghilangkan *missing value* dari kedua database.

In [ ]:
# Menghapus data pemesanan yang batal dimana nilai quantity nya negatif
# Menghapus data penjualan dari barang yang gratis
def dropNeg(df):
  df.drop(df[df['Quantity'] < 0].index, inplace = True)
  df.drop(df[df['Price'] == 0]. index, inplace = True)
  df.drop(df[df['Stock Code'] == "ADJUST"].index, inplace = True)

dropNeg(df1)
dropNeg(df2)


## Data Analysis

In [ ]:
def extractData(dataset, sheet, columns):
  global df
  df = pd.read_excel(dataset, sheet_name = sheet, usecols = columns)
  return df

#INI GAKEPAKE SETDATATYPE
def setDataType(df):
  df["Customer ID"] = df["Customer ID"].astype('string')
  df["Day"] = df["Day"].astype('string')
  df["Month"] = df["Month"].astype('string')
  df["Year"] = df["Year"].astype('string')

def setData(dataframe, columns):
  global df
  df = dataframe
  df['Day'] = pd.DatetimeIndex(df['Invoice Date']).day
  df['Month'] = pd.DatetimeIndex(df['Invoice Date']).month
  df['Year'] = pd.DatetimeIndex(df['Invoice Date']).year
  df.set_axis(['Invoice', 'Stock Code', 'Description', 'Quantity', 'Invoice Date', 'Price', 'Customer ID', 'Country', 'Day', 'Month', 'Year'], axis='columns', inplace=True)
  df = df[columns]
  return df





In [ ]:
#### Extract Dataset

dataCustomer1 = setData(df1, ["Quantity", "Country", "Customer ID", "Price"])
# dataProduct1 = setData(df1, ["Quantity", "Description", "Price", "Year", "Month", "Day"])
dataProduct1 = setData(df1, ["Description"])
dataSales1 = setData(df1, ["Quantity", "Description", "Price", "Year", "Month", "Day"])

#### Convert datatype for "Customer ID"
# setDataType(dataCustomer1)
# setDataType(dataSales1)
# setDataType(dataCustomer2)

#### Create new columns "Total Value"
dataCustomer1["Total Value"] = dataCustomer1["Quantity"]*dataCustomer1["Price"]
dataSales1["Total Value"] = dataSales1["Quantity"]*dataSales1["Price"]


<ipython-input-10-aef69038250a>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(['Invoice', 'Stock Code', 'Description', 'Quantity', 'Invoice Date', 'Price', 'Customer ID', 'Country', 'Day', 'Month', 'Year'], axis='columns', inplace=True)
<ipython-input-10-aef69038250a>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(['Invoice', 'Stock Code', 'Description', 'Quantity', 'Invoice Date', 'Price', 'Customer ID', 'Country', 'Day', 'Month', 'Year'], axis='columns', inplace=True)
<ipython-input-10-aef69038250a>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(['Invoice', 'Stock Code', 'Description', 'Quantity', 'Invoice

In [ ]:
#### Extract Dataset
dataCustomer2 = setData(df2, ["Quantity", "Country", "Customer ID", "Price"])
dataProduct2 = setData(df2, ["Description"])
dataSales2 = setData(df2, ["Quantity", "Description", "Price", "Year", "Month", "Day"])

#### Convert datatype for "Customer ID"
# setDataType(dataCustomer2)
# setDataType(dataSales2)

#### Create new columns "Total Value"
dataCustomer2["Total Value"] = dataCustomer2["Quantity"]*dataCustomer2["Price"]
dataSales2["Total Value"] = dataSales2["Quantity"]*dataSales2["Price"]

<ipython-input-10-aef69038250a>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(['Invoice', 'Stock Code', 'Description', 'Quantity', 'Invoice Date', 'Price', 'Customer ID', 'Country', 'Day', 'Month', 'Year'], axis='columns', inplace=True)
<ipython-input-10-aef69038250a>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(['Invoice', 'Stock Code', 'Description', 'Quantity', 'Invoice Date', 'Price', 'Customer ID', 'Country', 'Day', 'Month', 'Year'], axis='columns', inplace=True)
<ipython-input-10-aef69038250a>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(['Invoice', 'Stock Code', 'Description', 'Quantity', 'Invoice

In [ ]:
dataFrame_list = list()
dataFrame_desc = list()

def dfList(dataframe):
  dataFrame_list.append(df)
  return dataFrame_list

def dfDesc(description):
  dataFrame_desc.append(description)
  return dataFrame_desc

def top(data, col_reference, nrows, description):

  data["Total Value"] = data["Quantity"] * data["Price"]
  top = data.groupby(col_reference)["Quantity", "Total Value"].sum()
  tops = top.sort_values(by = ["Total Value"], ascending = False). iloc[:nrows,:]
  list = tops.index.tolist()
  # dataFrame_desc = dfDesc(description)
  return tops, list

def top2(data, col_reference, col_list, description):
  data["Total Value"] = data["Quantity"] * data["Price"]
  top = data.groupby(col_reference)["Quantity", "Total Value"].sum()
  tops = top.loc[col_list]
  # dataFrame_desc = dfDesc(description)
  return tops

# topPRODUCT = top(dataSales2, "Description", 11, "Top 10 best selling product in 2010-2011")
# dataFrame_list.append(topPRODUCT[0])
topCOUNTRY1 = top(dataCustomer1, "Country", 10, "Top 10 Country in 2009-2010")
topCOUNTRY2 = top(dataCustomer2, "Country", 10, "Top 10 Country in 2010-2011")
# dataFrame_list.append(topCOUNTRY[0])
# topCUSTOMERS = top(dataCustomer2, "Customer ID", 11, "Top 10 Customer in 2010-2011")
# dataFrame_list.append(topCUSTOMERS[0])

#### Top 10 in 2009-2010
# topCountry1 = top(dataCustomer1,"Country",11, "Top 10 countries with largest purchases in 2009-2010")
# dataFrame_list.append(topCountry1[0])
# topCountry2 = top2(dataCustomer2, "Country", topCountry1[1], "Top 10 countries with largest purchases in 2010-2011")
# dataFrame_list.append(topCountry2)




# topCustomer1 = top(dataCustomer1, "Customer ID", 11, "Top 10 customers with largest purchases in 2009-2010")
# dataFrame_list.append(topCustomer1[0])
# topCustomer2 = top2(dataCustomer2, "Customer ID", topCustomer1[1], "Top 10 customers with largest purchases in 2010-2011")
# dataFrame_list.append(topCustomer2)

# topSales1 = top(dataSales1, "Description", 11, "Top 10 best selling products in 2009-2010")
# dataFrame_list.append(topSales1[0])
# topSales2 = top2(dataSales2, "Description", topSales1[1], "Top 10 best selling products in 2010-2011")
# dataFrame_list.append(topSales2)

# topMonth1 = top(dataSales1, "Month", 12, "Revenue per Month in 2009-2010")
# dataFrame_list.append(topMonth1[0])

# topMonth2 = top2(dataSales2, "Month", topMonth1[1], "Revenue per Month in 2010-2011")
# dataFrame_list.append(topMonth2)



print(topCOUNTRY1[1])

print(topCOUNTRY2[1])


# #### Top 10 in 2010-2011
# topCountry2 = top(dataCustomer2,"Country",10, "Top 10 countries with largest purchases in 2010-2011")
# topCustomer2 = top(dataCustomer2, "Customer ID", 10, "Top 10 customers with largest purchases in 2010-2011")
# topSales2 = top(dataSales2, "Description", 10, "Top 10 best selling products in 2010-2011")
# topMonth2 = top(dataSales2, "Month", 12, "Months with largest revenue in 2010-2011")






['United Kingdom', 'EIRE', 'Netherlands', 'Germany', 'France', 'Sweden', 'Denmark', 'Spain', 'Switzerland', 'Australia']
['United Kingdom', 'Netherlands', 'EIRE', 'Germany', 'France', 'Australia', 'Spain', 'Switzerland', 'Belgium', 'Sweden']


<ipython-input-59-ce94e8d025f1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Total Value"] = data["Quantity"] * data["Price"]
<ipython-input-59-ce94e8d025f1>:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  top = data.groupby(col_reference)["Quantity", "Total Value"].sum()
<ipython-input-59-ce94e8d025f1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Total Value"] = data["Quantity"] * data["Price"]
<ipython

In [ ]:
# df_filtered1 = df[df['DESC'] == True].index
#   df.drop(df_filtered1, inplace = True)
#   df.drop('DESC', inplace = True, axis = 1)
dataFrame_list = list()
dataFrame_desc = list()

def dfList(dataframe):
  dataFrame_list.append(df)
  return dataFrame_list

def dfDesc(description):
  dataFrame_desc.append(description)
  return dataFrame_desc

# df.drop(df[df['Quantity'] < 0].index, inplace = True)
def topS(data, col_reference, nrows, description):
  data.drop(data[data[col_reference] == "PACK OF 72 RETRO SPOT CAKE CASES"]. index, inplace = True)
  data.drop(data[data[col_reference] == "BLACK AND WHITE PAISLEY FLOWER MUG"]. index, inplace = True)
  # filter1 = data[data[col_reference] == "PACK OF 72 RETRO SPOT CAKE CASES"]
  # filter2 = data[data[col_reference] == "BLACK AND WHITE PAISLEY FLOWER MUG"]
  # data.drop(filter1, inplace = True)
  # data.drop(filter2, inplace = True)
  data["Total Value"] = data["Quantity"] * data["Price"]
  top = data.groupby(col_reference)["Quantity", "Total Value"].sum()
  tops = top.sort_values(by = ["Quantity"], ascending = False). iloc[:nrows,:]
  list = tops.index.tolist()
  dataFrame_desc = dfDesc(description)
  return tops, list

def topS2(data, col_reference, col_list, description):
  data["Total Value"] = data["Quantity"] * data["Price"]
  top = data.groupby(col_reference)["Quantity", "Total Value"].sum()
  tops = top.loc[col_list]
  dataFrame_desc = dfDesc(description)
  return tops

topSales1 = topS(dataSales1, "Description", 11, "Top 10 best selling products in 2009-2010")
dataFrame_list.append(topSales1[0])
topSales2 = topS2(dataSales2, "Description", topSales1[1], "Top 10 best selling products in 2010-2011")
dataFrame_list.append(topSales2)
# print(topSales1)
# print(topSales2)
# dataSales1["Description"].head()

<ipython-input-32-1868149128a2>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(data[data[col_reference] == "PACK OF 72 RETRO SPOT CAKE CASES"]. index, inplace = True)
<ipython-input-32-1868149128a2>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(data[data[col_reference] == "BLACK AND WHITE PAISLEY FLOWER MUG"]. index, inplace = True)
<ipython-input-32-1868149128a2>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

In [ ]:
#### Total Revenue 2009-2010 and 2010-2011
# dataFrame_list.pop()
# dataFrame_desc.pop()
revenue1 = dataSales1["Total Value"].sum()
revenue2 = dataSales2["Total Value"].sum()
quantity1 = dataSales1["Quantity"].sum()
quantity2 = dataSales2["Quantity"].sum()
revenue = {'Year': ['Year 2009-2010', 'Year 2010-2011'],
        'Revenue': [revenue1, revenue2],
        'Quantity' : [quantity1, quantity2]}
totalRevenue = pd.DataFrame(revenue)
dataFrame_list.append(totalRevenue)
dataFrame_desc.append("Total Revenue 2009-2010 and 2010-2011")

# print(totalRevenue)

In [ ]:
#### Weeks of purchases in 2009-2010 and 2010-2011
dataWeek1 = setData(df1, ["Country","Quantity", "Description", "Price", "Year", "Month", "Day"])
dataWeek2 = setData(df2, ["Country","Quantity", "Description", "Price", "Year", "Month", "Day"])

def weekPurchases(df):

  conditions = [
      (df['Day'] <= 10),
      (df['Day'] > 10) & (df['Day'] <= 20),
      (df['Day'] > 20) & (df['Day'] <= 31),
      ]
  values = ['Early month', 'Mid month', 'Late month']

  df['Week of purchases'] = np.select(conditions, values)
  df["Total Value"] = df["Quantity"]*df["Price"]

  # if df is dataWeek1:
  #   countrySort = topCountry1[2]
  # elif df is dataWeek2:
    # countrySort = topCountry2.index.tolist()

  countrySort = topCOUNTRY2[1]
  list = []
  for country in countrySort:
    for i in values:
      tuple = (country, i)
      list.append(tuple)

  tops = df.groupby(["Country","Week of purchases"])["Quantity", "Total Value"].sum()
  tops = tops.sort_values(by = "Total Value", ascending = False)
  output = tops.reindex(list)
  return output

# weekYear1 = weekPurchases(dataWeek1)
# dataFrame_list.append(weekYear1)
# dataFrame_desc.append("Week of Purchases based on country in 2009-2010")

weekYear2 = weekPurchases(dataWeek2)
dataFrame_list.append(weekYear2)
dataFrame_desc.append("Week of Purchases based on country in 2010-2011")

# print(weekYear2)


<ipython-input-10-aef69038250a>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(['Invoice', 'Stock Code', 'Description', 'Quantity', 'Invoice Date', 'Price', 'Customer ID', 'Country', 'Day', 'Month', 'Year'], axis='columns', inplace=True)
<ipython-input-10-aef69038250a>:19: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(['Invoice', 'Stock Code', 'Description', 'Quantity', 'Invoice Date', 'Price', 'Customer ID', 'Country', 'Day', 'Month', 'Year'], axis='columns', inplace=True)
<ipython-input-61-b21fe6f1e8fa>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [ ]:
def holidayStats(df, holiday):
  if holiday.lower() == "christmas":
    data = df.loc[(df["Month"] == 12) & (df["Day"] >= 1) & (df["Day"] <= 24)]
    sumQ = data["Quantity"].sum()
    sumT = data["Total Value"].sum()

  elif holiday.lower() == "chinese":
    if df is dataSales1:
      data = df.loc[((df["Month"] == 1) & (df["Day"] >= 14)) | ((df["Month"] == 2) & (df["Day"] <= 14))]
    else:
      data = df.loc[((df["Month"] == 1) & (df["Day"] >= 3)) | ((df["Month"] == 2) & (df["Day"] <= 3))]
    sumQ = data["Quantity"].sum()
    sumT = data["Total Value"].sum()

  elif holiday.lower() == "ramadan":
    if df is dataSales1:
      data = df.loc[((df["Month"] == 8) & (df["Day"] >= 11)) | ((df["Month"] == 9) & (df["Day"] <= 9))]
    else:
      data = df.loc[(df["Month"] == 8) & (df["Day"] >= 1) & (df["Day"] <= 30)]
    sumQ = data["Quantity"].sum()
    sumT = data["Total Value"].sum()

  elif holiday.lower() == "valentine":
    data = df.loc[((df["Month"] == 1) & (df["Day"] >= 14)) | ((df["Month"] == 2) & (df["Day"] <= 14))]
    sumQ = data["Quantity"].sum()
    sumT = data["Total Value"].sum()

  elif holiday.lower() == "easter":
    if df is dataSales1:
      data = df.loc[((df["Month"] == 3) & (df["Day"] >= 4)) | ((df["Month"] == 4) & (df["Day"] <= 4))]
    else:
      data = df.loc[((df["Month"] == 3) & (df["Day"] >= 24)) | ((df["Month"] == 4) & (df["Day"] <= 24))]
    sumQ = data["Quantity"].sum()
    sumT = data["Total Value"].sum()

  return sumQ, sumT

christmas1 = holidayStats(dataSales1, 'christmas')
chinese1 = holidayStats(dataSales1, 'chinese')
ramadan1 = holidayStats(dataSales1, 'ramadan')
valentine1 = holidayStats(dataSales1, 'valentine')
easter1 = holidayStats(dataSales1, 'easter')
data = {"Holiday" : ["Christmas", "Chinese New Year", "Ramadan", "Valentine's Day", "Easter Day"],
        "Total Quantity" : [christmas1[0], chinese1[0], ramadan1[0], valentine1[0], easter1[0]],
        "Total Value" : [christmas1[1], chinese1[1], ramadan1[1], valentine1[1], easter1[1]]}
dataHoliday1 = pd.DataFrame(data)
dataFrame_list.append(dataHoliday1)
dataFrame_desc.append("Sales Trend on Public Holidays 2009-2010")

christmas2 = holidayStats(dataSales2, 'christmas')
chinese2 = holidayStats(dataSales2, 'chinese')
ramadan2 = holidayStats(dataSales2, 'ramadan')
valentine2 = holidayStats(dataSales2, 'valentine')
easter2 = holidayStats(dataSales2, 'easter')
data = {"Holiday" : ["Christmas", "Chinese New Year", "Ramadan", "Valentine's Day", "Easter Day"],
        "Total Quantity" : [christmas2[0], chinese2[0], ramadan2[0], valentine2[0], easter2[0]],
        "Total Value" : [christmas2[1], chinese2[1], ramadan2[1], valentine2[1], easter2[1]]}
dataHoliday2 = pd.DataFrame(data)
dataFrame_list.append(dataHoliday2)
dataFrame_desc.append("Sales Trend on Public Holidays 2010-2011")


In [ ]:
#### Top 10 products with increasing revenue
# df1.merge(df2[['Name', 'Grade', 'Rank']])
# df1 = dataProduct1
df1 = dataSales1[["Description", "Quantity", "Total Value"]]
# df2 = dataSales2[["Description", "Quantity", "Total Value"]]
df1.head()
# df2.head()

# subs = df2["Total Value"] - df1["Total Value"]
# subs.head()

# Problem:
# Can't merge dataframe because it's too large
# Expectation: Product A has X revenue on 09-10 and Y revenue on 10-11
# Expectation: Country A has X purchases on 09-10 and Y purchases on 10-11
# Expectation: Customer A has X purchases on 09-10 and Y purchases on 10-11
#### Top 10 products with decreasing revenue

,Description,Quantity,Total Value
0,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,83.4
1,PINK CHERRY LIGHTS,12,81.0
2,WHITE CHERRY LIGHTS,12,81.0
3,"RECORD FRAME 7"" SINGLE SIZE",48,100.8
4,STRAWBERRY CERAMIC TRINKET BOX,24,30.0


In [ ]:
directory = r'/content/gdrive/MyDrive/Semester 6/Kapsel Statistika/UTS Output.xlsx'

In [ ]:
print('total value')
topp = dataCustomer1.groupby("Customer ID")["Quantity", "Total Value"].sum()
# topi2 = topi.sort_values(by = ["Total Value"], ascending = False). iloc[:500000,:]
topp
# topi2.loc[(topi2['Total Value'] > 0) & (topi2['Total Value'] < 307) ]
# topi2['Total Value'].describe()
# print("\n")
# print('quantity')
# dataCustomer1['Quantity'].describe()
# print('price')
# dataCustomer1['Price'].describe()

total value


<ipython-input-59-9dba32a2f8e0>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  topi = dataCustomer1.groupby("Customer ID")["Quantity", "Total Value"].sum()


,Quantity,Total Value
Customer ID,,
12346.0,70,372.86
12347.0,828,1323.32
12348.0,373,222.16
12349.0,993,2671.14
12351.0,261,300.93
...,...,...
18283.0,336,641.77
18284.0,494,461.68
18285.0,145,427.00


In [ ]:
# topp['Total Value'].describe()
topp['Quantity'].describe()

count      4312.000000
mean       1284.404917
std        6459.164580
min           1.000000
25%         158.000000
50%         382.000000
75%         996.250000
max      220600.000000
Name: Quantity, dtype: float64

In [ ]:
topp['Total Value'].describe()

count      4302.000000
mean       2052.176837
std        8919.179645
min           2.950000
25%         309.242500
50%         710.690000
75%        1731.402500
max      349164.350000
Name: Total Value, dtype: float64

In [ ]:
# group1 = dataCustomer1.groupby("Customer ID")["Quantity", "Total Value"].sum()
# group2 = dataCustomer2.groupby("Customer ID")["Quantity", "Total Value"].sum()

# def groupCustomers(df, category):
#   if category == "Total Value":
#     q1 = df[category].quantile(0.25)
#     q2 = df[category].quantile(0.5)
#     q3 = df[category].quantile(0.75)
#     max = df[category].quantile(1)

#     conditions = [
#       (df[category] <= q1),
#       (df[category] > q1) & (df[category] <= q2),
#       (df[category] > q2) & (df[category] <= q3),
#       (df[category] > q3) & (df[category] <= max),
#     ]

#     values = ['Low spending', 'Medium spending', 'High spending', 'Very high spending']
#     df['Spending level'] = np.select(conditions, values)

#     lowS = df['Spending level'].value_counts()['Low spending']
#     midS = df['Spending level'].value_counts()['Medium spending']
#     highS = df['Spending level'].value_counts()['High spending']
#     vhighS = df['Spending level'].value_counts()['Very high spending']

#     numCustomers = [lowS, midS, highS, vhighS]

#   elif category == "Quantity":
#     q1 = df[category].quantile(0.25)
#     q2 = df[category].quantile(0.5)
#     q3 = df[category].quantile(0.75)
#     max = df[category].quantile(1)

#     conditions = [
#       (df[category] <= q1),
#       (df[category] > q1) & (df[category] <= q2),
#       (df[category] > q2) & (df[category] <= q3),
#       (df[category] > q3) & (df[category] <= max),]

#     values = ['Low purchase', 'Medium purchase', 'High purchase', 'Very high purchase']
#     df['Purchase level'] = np.select(conditions, values)

#     lowP = df['Purchase level'].value_counts()['Low purchase']
#     midP = df['Purchase level'].value_counts()['Medium purchase']
#     highP = df['Purchase level'].value_counts()['High purchase']
#     vhighP = df['Purchase level'].value_counts()['Very high purchase']

#     numCustomers = [lowP, midP, highP, vhighP]

#   summary = {"Category" : values,
#            "Range": ["Less than " + str(q1), str(q1) + " - " + str(q2), str(q2) + " - " + str(q3), "More than " + str(q3)],
#            "Number of Customers" : numCustomers}
#   summary = pd.DataFrame(summary)
#   return summary

# dataFrame_list.append(summary1)
# dataFrame_desc.append("Group of Customers based on Spending Level")
# dataFrame_list.append(summary2)
# dataFrame_desc.append("Group of Customers based on Purchase Level")

# summary1_V = groupCustomers(group1, "Total Value")
# dataFrame_list.append(summary1_V)
# dataFrame_desc.append("Group of Customers based on Spending Level in 2009-2010")

# summary1_Q = groupCustomers(group1, "Quantity")
# dataFrame_list.append(summary1_Q)
# dataFrame_desc.append("Group of Customers based on Purchase Level in 2009-2010")

# summary2_V = groupCustomers(group2, "Total Value")
# dataFrame_list.append(summary2_V)
# dataFrame_desc.append("Group of Customers based on Spending Level in 2010-2011")

# summary2_Q = groupCustomers(group2, "Quantity")
# dataFrame_list.append(summary2_Q)
# dataFrame_desc.append("Group of Customers based on Purchase Level in 2010-2011")


<ipython-input-55-96e4ef38a903>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  group1 = dataCustomer1.groupby("Customer ID")["Quantity", "Total Value"].sum()
<ipython-input-55-96e4ef38a903>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  group2 = dataCustomer2.groupby("Customer ID")["Quantity", "Total Value"].sum()


In [ ]:
group1 = dataCustomer1.groupby("Customer ID")["Quantity", "Total Value"].sum()
group2 = dataCustomer2.groupby("Customer ID")["Quantity", "Total Value"].sum()

def groupCustomers(df, category):
  if category == "Total Value":
    q1 = df[category].quantile(0.25)
    q2 = df[category].quantile(0.5)
    q3 = df[category].quantile(0.75)
    max = df[category].quantile(1)

    conditions = [
      (df[category] <= q1),
      (df[category] > q1) & (df[category] <= q2),
      (df[category] > q2) & (df[category] <= q3),
      (df[category] > q3) & (df[category] <= max),
    ]

    values = ['Low spending', 'Medium spending', 'High spending', 'Very high spending']
    df['Spending level'] = np.select(conditions, values)

    lowS = df['Spending level'].value_counts()['Low spending']
    midS = df['Spending level'].value_counts()['Medium spending']
    highS = df['Spending level'].value_counts()['High spending']
    vhighS = df['Spending level'].value_counts()['Very high spending']

    numCustomers = [lowS, midS, highS, vhighS]

    meanQ = []
    for i in values:
      mean = df.loc[df['Spending level'] == i].mean()[0]
      meanQ.append(mean)

  elif category == "Quantity":
    q1 = df[category].quantile(0.25)
    q2 = df[category].quantile(0.5)
    q3 = df[category].quantile(0.75)
    max = df[category].quantile(1)

    conditions = [
      (df[category] <= q1),
      (df[category] > q1) & (df[category] <= q2),
      (df[category] > q2) & (df[category] <= q3),
      (df[category] > q3) & (df[category] <= max),]

    values = ['Low purchase', 'Medium purchase', 'High purchase', 'Very high purchase']
    df['Purchase level'] = np.select(conditions, values)

    lowP = df['Purchase level'].value_counts()['Low purchase']
    midP = df['Purchase level'].value_counts()['Medium purchase']
    highP = df['Purchase level'].value_counts()['High purchase']
    vhighP = df['Purchase level'].value_counts()['Very high purchase']

    numCustomers = [lowP, midP, highP, vhighP]

    meanV = []
    for i in values:
      mean = df.loc[df['Purchase level'] == i].mean()[1]
      meanV.append(mean)

  summary = {"Category" : values,
           "Range": ["Less than " + str(q1), str(q1) + " - " + str(q2), str(q2) + " - " + str(q3), "More than " + str(q3)],
           "Number of Customers" : numCustomers}

  if category == "Total Value":
    summary['Mean of Quantity Purchased'] = meanQ
  elif category == "Quantity":
    summary['Mean of Total Spending'] = meanV

  summary = pd.DataFrame(summary)
  return summary

# groupCustomers(group1, "Quantity")

summary1_V = groupCustomers(group1, "Total Value")
dataFrame_list.append(summary1_V)
dataFrame_desc.append("Group of Customers based on Spending Level in 2009-2010")

summary1_Q = groupCustomers(group1, "Quantity")
dataFrame_list.append(summary1_Q)
dataFrame_desc.append("Group of Customers based on Purchase Level in 2009-2010")

summary2_V = groupCustomers(group2, "Total Value")
dataFrame_list.append(summary2_V)
dataFrame_desc.append("Group of Customers based on Spending Level in 2010-2011")

summary2_Q = groupCustomers(group2, "Quantity")
dataFrame_list.append(summary2_Q)
dataFrame_desc.append("Group of Customers based on Purchase Level in 2010-2011")

<ipython-input-20-7cc65510ec29>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  group1 = dataCustomer1.groupby("Customer ID")["Quantity", "Total Value"].sum()
<ipython-input-20-7cc65510ec29>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  group2 = dataCustomer2.groupby("Customer ID")["Quantity", "Total Value"].sum()
<ipython-input-20-7cc65510ec29>:30: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  mean = df.loc[df['Spending level'] == i].mean()[0]
<ipython-input-20-7cc65510ec29>:30: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will defaul

In [ ]:

data1 = dataSales1.loc[(dataSales1["Description"].str.contains(r"CHRISTMAS"))]
data1 = data1[["Description", "Quantity", "Total Value"]]

data2 = dataSales2.loc[(dataSales2["Description"].str.contains(r"CHRISTMAS"))]
data2 = data2[["Description", "Quantity", "Total Value"]]

def topChristmas(df, nrows):
  top = df.groupby("Description")["Quantity", "Total Value"].sum()
  top = top.sort_values(by = ["Quantity"], ascending = False). iloc[:nrows,:]
  list = top.index.tolist()
  return top, list

def topChristmas2(df, col_list):
  top = df.groupby("Description")["Quantity", "Total Value"].sum()
  tops = top.loc[col_list]
  return tops

# christmas1 = topChristmas(data1, 10)
# dataFrame_list.append(christmas1[0])
# dataFrame_desc.append("Top 10 Christmas Products in 2009-2010")

# christmas2 = topChristmas2(data2, christmas1[1])

# dataFrame_list.append(christmas2)
# dataFrame_desc.append("Top 10 Christmas Products in 2010-2011")

topCHRISTMAS = topChristmas(data2, 10)
dataFrame_list.append(topCHRISTMAS[0])
dataFrame_desc.append("Top 10 Christmas Products in 2010-2011")


<ipython-input-46-e2db8c15fb3c>:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  top = df.groupby("Description")["Quantity", "Total Value"].sum()


In [ ]:
df1["Country"].unique()

array(['United Kingdom', 'France', 'USA', 'Belgium', 'Australia', 'EIRE',
       'Germany', 'Portugal', 'Denmark', 'Netherlands', 'Poland',
       'Channel Islands', 'Spain', 'Cyprus', 'Greece', 'Norway',
       'Austria', 'Sweden', 'United Arab Emirates', 'Finland', 'Italy',
       'Switzerland', 'Japan', 'Unspecified', 'Nigeria', 'Malta', 'RSA',
       'Singapore', 'Bahrain', 'Thailand', 'Israel', 'Lithuania',
       'West Indies', 'Korea', 'Brazil', 'Canada', 'Iceland'],
      dtype=object)

In [ ]:
def holidayPurchases(holiday, category):
  # if holiday.lower() == "christmas":
  #   df = dataSales1.loc[(dataSales1["Month"] == 12) & (dataSales1["Day"] >= 11) & (dataSales1["Day"] <= 24)]
  #   df = df.groupby(dataProduct1[category])["Quantity", "Total Value"].sum()
  #   df = df.sort_values(by = ["Quantity"], ascending = False). iloc[:10,:]
  # elif holiday.lower() == "chinese":
  #   df = dataSales1.loc[(dataSales1["Month"] == 1) & (dataSales1["Day"] >= 1) & (dataSales1["Day"] <= 26)]
  #   df = df.groupby(dataProduct1[category])["Quantity", "Total Value"].sum()
  #   df = df.sort_values(by = ["Quantity"], ascending = False). iloc[:20,:]
  # elif holiday.lower() == "ramadan":
  #   df = dataSales1.loc[((dataSales1["Month"] == 8) & (dataSales1["Day"] >= 22)) | ((dataSales1["Month"] == 9) & (dataSales1["Day"] <= 20))]
  #   df = df.groupby(dataProduct1[category])["Quantity", "Total Value"].sum()
  #   df = df.sort_values(by = ["Total Value"], ascending = False). iloc[:10,:]
  # return df
  global sumQ, sumT
  if holiday.lower() == "christmas":
    df = dataSales1.loc[(dataSales1["Month"] == 12) & (dataSales1["Day"] >= 11) & (dataSales1["Day"] <= 24)]
    sumQ = df["Quantity"].sum()
    sumT = df["Total Value"].sum()
    # df = df.groupby(["Quantity", "Total Value"].sum())
    # df = df.sort_values(by = ["Quantity"], ascending = False). iloc[:10,:]
  elif holiday.lower() == "chinese":
    df = dataSales1.loc[(dataSales1["Month"] == 1) & (dataSales1["Day"] >= 1) & (dataSales1["Day"] <= 26)]
    df = df.groupby(dataProduct1[category])["Quantity", "Total Value"].sum()
    df = df.sort_values(by = ["Quantity"], ascending = False). iloc[:20,:]
  elif holiday.lower() == "ramadan":
    df = dataSales1.loc[((dataSales1["Month"] == 8) & (dataSales1["Day"] >= 22)) | ((dataSales1["Month"] == 9) & (dataSales1["Day"] <= 20))]
    df = df.groupby(dataProduct1[category])["Quantity", "Total Value"].sum()
    df = df.sort_values(by = ["Total Value"], ascending = False). iloc[:10,:]
  return sumQ, sumT



# a = holidayPurchases('christmas', 'Description')
# print(a)
# print(sumQ, sumT)
# a.describe()




In [ ]:
df = dataProduct1[dataProduct1["Description"] == "Manual"]
df

,Quantity,Description,Price,Year,Month,Day
11310,1,Manual,0.85,2009,12,4
11311,1,Manual,0.21,2009,12,4
17386,1,Manual,10.00,2009,12,8
17887,1,Manual,10.00,2009,12,8
19542,1,Manual,15.95,2009,12,8
...,...,...,...,...,...,...
516892,2,Manual,5.95,2010,12,6
521074,12,Manual,0.19,2010,12,8
521122,12,Manual,2.95,2010,12,8
521124,12,Manual,0.19,2010,12,8


In [ ]:
# dataFrame_desc.pop()
# dataFrame_list.pop()
print(len(dataFrame_list))
counter = 0
for i in dataFrame_desc:
  print(counter, i)
  counter +=1

2
0 Week of Purchases based on country in 2009-2010
1 Week of Purchases based on country in 2010-2011


In [ ]:
# dataFrame_list = []
# dataFrame_desc = []

In [ ]:
def load(dfList, dfDesc):
  with pd.ExcelWriter(r'/content/gdrive/MyDrive/Semester 6/Kapsel Statistika/Kapselstat Output.xlsx') as writer:
    for i in range(len(dfList)):
      dfList[i].to_excel(writer, sheet_name = "Sheet" + str(i), startrow = 3)
      work = writer.book
      worksheet = writer.sheets['Sheet' + str(i)]
      worksheet.write(0,0, dfDesc[i])

load(dataFrame_list, dataFrame_desc)

NameError: ignored

In [ ]:
list = ["Top 10 best selling product in 2009-2010",
"Top 10 best selling product in 2010-2011",
"Data of 2009-2010 best selling product in two time periods",
"Top 10 best christmas product in 2009-2010",
"Top 10 best christmas product in 2010-2011",
"Data of 2009-2010 best christmas product in two time periods",
"Top 10 countries with largest purchases in 2009-2010",
"Top 10 countries with largest purchases in 2010-2011",
"Data of 2009-2010 top countries in two time periods",
"Top 10 customers with largest purchases in 2009-2010",
"Top 10 customers with largest purchases in 2010-2011",
"Data of 2009-2010 top customers in two time periods",
"Group of Customers based on Spending Level in 2009-2010",
"Group of Customers based on Spending Level in 2010-2011",
"Group of Customers based on Purchase Level in 2009-2010",
"Group of Customers based on Purchase Level in 2010-2011",
"Total Revenue per Months in 2009-2010",
"Total Revenue per Months in 2010-2011",
"Quantity and Total Revenue in two time periods",
"Week of Purchases based on country in 2009-2010",
"Week of Purchases based on country in 2010-2011",
"Sales Trend on Public Holidays in 2009-2010",
"Sales Trend on Public Holidays in 2010-2011"]
counter = 1
for i in list:
  print(counter, i)
  counter += 1

1 Top 10 best selling product in 2009-2010
2 Top 10 best selling product in 2010-2011
3 Data of 2009-2010 best selling product in two time periods
4 Top 10 best christmas product in 2009-2010
5 Top 10 best christmas product in 2010-2011
6 Data of 2009-2010 best christmas product in two time periods
7 Top 10 countries with largest purchases in 2009-2010
8 Top 10 countries with largest purchases in 2010-2011
9 Data of 2009-2010 top countries in two time periods
10 Top 10 customers with largest purchases in 2009-2010
11 Top 10 customers with largest purchases in 2010-2011
12 Data of 2009-2010 top customers in two time periods
13 Group of Customers based on Spending Level in 2009-2010
14 Group of Customers based on Spending Level in 2010-2011
15 Group of Customers based on Purchase Level in 2009-2010
16 Group of Customers based on Purchase Level in 2010-2011
17 Total Revenue per Months in 2009-2010
18 Total Revenue per Months in 2010-2011
19 Quantity and Total Revenue in two time periods
2